<a href="https://colab.research.google.com/github/harrisontg57/NLP_Spring_2021/blob/main/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tim Harrison ID: 002-24-5948

In [132]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Problem 1)

In [78]:
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



play_names = ["a-midsummer-nights-dream","alls-well-that-ends-well","antony-and-cleopatra","as-you-like-it","coriolanus","cymbeline","hamlet","henry-iv-part-1","henry-iv-part-2","henry-v","henry-vi-part-1","henry-vi-part-2","henry-vi-part-3","henry-viii","julius-caesar"]
play_strings = []

for title in play_names:
  with open('/content/'+title+'_TXT_FolgerShakespeare.txt', 'r') as file:
      d = file.read().replace('\n', ' ')
      play_strings.append(d)

#print(len(play_strings))

Tfidf_vect = TfidfVectorizer()
vector_matrix = Tfidf_vect.fit_transform(play_strings)

cosine_similarity_matrix = cosine_similarity(vector_matrix)
#print(cosine_similarity_matrix)
pd.DataFrame(data=cosine_similarity_matrix,index=play_names,columns=play_names)

,a-midsummer-nights-dream,alls-well-that-ends-well,antony-and-cleopatra,as-you-like-it,coriolanus,cymbeline,hamlet,henry-iv-part-1,henry-iv-part-2,henry-v,henry-vi-part-1,henry-vi-part-2,henry-vi-part-3,henry-viii,julius-caesar
a-midsummer-nights-dream,1.000000,0.721492,0.649114,0.665538,0.688681,0.718656,0.677110,0.760838,0.768654,0.771913,0.743208,0.760478,0.719175,0.769703,0.655901
alls-well-that-ends-well,0.721492,1.000000,0.683257,0.695009,0.734359,0.762164,0.716698,0.783370,0.805714,0.798095,0.768704,0.782100,0.740178,0.829208,0.680604
antony-and-cleopatra,0.649114,0.683257,1.000000,0.610599,0.659735,0.678471,0.630071,0.700240,0.713721,0.709946,0.686683,0.702209,0.664561,0.723164,0.711354
as-you-like-it,0.665538,0.695009,0.610599,1.000000,0.661847,0.685904,0.636910,0.710177,0.724722,0.717780,0.687543,0.701415,0.660897,0.732286,0.621824
coriolanus,0.688681,0.734359,0.659735,0.661847,1.000000,0.727801,0.675620,0.745189,0.768706,0.762026,0.725305,0.741927,0.700714,0.788711,0.691778
cymbeline,0.718656,0.762164,0.678471,0.685904,0.727801,1.000000,0.701331,0.776533,0.788278,0.781753,0.758870,0.775450,0.734600,0.807625,0.675647
hamlet,0.677110,0.716698,0.630071,0.636910,0.675620,0.701331,1.000000,0.738146,0.753435,0.756079,0.720225,0.740773,0.702772,0.768048,0.632892
henry-iv-part-1,0.760838,0.783370,0.700240,0.710177,0.745189,0.776533,0.738146,1.000000,0.928976,0.862504,0.824907,0.842985,0.798493,0.843569,0.707524
henry-iv-part-2,0.768654,0.805714,0.713721,0.724722,0.768706,0.788278,0.753435,0.928976,1.000000,0.879595,0.828876,0.852334,0.810499,0.870072,0.718555
henry-v,0.771913,0.798095,0.709946,0.717780,0.762026,0.781753,0.756079,0.862504,0.879595,1.000000,0.865623,0.881874,0.838788,0.876847,0.717541


Problem 2)

In [131]:
import numpy as np

def top_n(m,n):
  li = []
  i = np.identity(len(play_names))
  df =  pd.DataFrame(data=m,index=play_names,columns=play_names)
  ident = pd.DataFrame(data=i,index=play_names,columns=play_names)
  no_ones = df.sub(ident)
  for k in range(n):
    top = np.argmax(no_ones)
    li.append((play_names[int(top/len(play_names))],play_names[top%len(play_names)]))
    no_ones.at[play_names[int(top/len(play_names))],play_names[top%len(play_names)]] = 0
    no_ones.at[play_names[top%len(play_names)],play_names[int(top/len(play_names))]] = 0
  return li
top_n(cosine_similarity_matrix,10)

[('henry-iv-part-1', 'henry-iv-part-2'),
 ('henry-vi-part-1', 'henry-vi-part-2'),
 ('henry-vi-part-2', 'henry-vi-part-3'),
 ('henry-v', 'henry-vi-part-2'),
 ('henry-iv-part-2', 'henry-v'),
 ('henry-v', 'henry-viii'),
 ('henry-iv-part-2', 'henry-viii'),
 ('henry-vi-part-2', 'henry-viii'),
 ('henry-v', 'henry-vi-part-1'),
 ('henry-iv-part-1', 'henry-v')]

Problem 3)

In [80]:
import nltk
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
nltk.download('punkt')

model = defaultdict(lambda: defaultdict(lambda: 0))

plays = []

for title in play_names:
  with open('/content/'+title+'_TXT_FolgerShakespeare.txt', 'r') as file:
    lines = [line.rstrip() for line in file]
    #remove non letters except '
    lines = [re.sub(r'[^a-zA-Z0-9_\']',' ',line) for line in lines]
    #Remove Capitalized words (ie character names)
    lines = [re.sub(r'[A-Z][A-Z]+','',line) for line in lines]
    plays.extend(lines)
#print(plays)

model_t = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in plays:
    #print(nltk.word_tokenize(sentence))
    for w1, w2, w3 in trigrams(nltk.word_tokenize(sentence), pad_right=True, pad_left=True):
        model_t[(w1, w2)][w3] += 1

for w1_w2 in model_t:
    total_count = float(sum(model_t[w1_w2].values()))
    for w3 in model_t[w1_w2]:
        model_t[w1_w2][w3] /= total_count

#dict(model_t["the","time"])

model_b = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in plays:
    #print(nltk.word_tokenize(sentence))
    for w1, w2 in bigrams(nltk.word_tokenize(sentence), pad_right=True, pad_left=True):
        model_b[(w1)][w2] += 1

for w1 in model_b:
    total_count = float(sum(model_b[w1].values()))
    for w2 in model_b[w1]:
        model_b[w1][w2] /= total_count

#dict(model_b["dog"])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Problem 4)

In [102]:
import random

# starting words

def sentence_generator(model, start_terms, number_sentences):
  text = start_terms[:]
  new_sentences = []
  if model == "trigram":
    for x in range(number_sentences):
        sentence_finished = False
        while not sentence_finished:
          # select a random probability threshold  
          r = random.random()
          accumulator = .0
          for word in model_t[tuple(text[-2:])].keys():
              accumulator += model_t[tuple(text[-2:])][word]
              # select words that are above the probability threshold
              if accumulator >= r:
                  text.append(word)
                  break
          if text[-2:] == [None, None]:
              sentence_finished = True
        new_sentences.append(' '.join([t for t in text if t]))
        text = start_terms[:]

  elif model == "bigram":
    for x in range(number_sentences):
        sentence_finished = False
        while not sentence_finished:
          # select a random probability threshold  
          r = random.random()
          accumulator = .0
          for word in model_b[text[-1]].keys():
              accumulator += model_b[text[-1]][word]
              # select words that are above the probability threshold
              if accumulator >= r:
                  text.append(word)
                  break
          if text[-2:] == [None, None]:
              sentence_finished = True
        new_sentences.append(' '.join([t for t in text if t]))
        text = start_terms[:]
  return new_sentences

print(sentence_generator("bigram",["come"],10))
print(sentence_generator("trigram",["as", "much"],5))

['come to the walls Pucelle now lady nigh Enter a poor soul In private malice of rebellion', "come what prevention He is the world see Titan and much and if I would fain shut us what hope sir are my eye and that he is gone The ragged'st hour of the like larks", 'come to Buckingham on And Britons cold and read it for men I sent lovers love Rosalind these wars', 'come you I am hushed with the spirit will go in the writs go But I grow dull If it for the law thou live Is your places But take their knavery will look you would not find Tut I will blow away to break his good cousin Shallow performs', "come She shall be For I can that no I By the top proud Where now Cominius head on you prove an exact command the French lack some more trust thee poetical with joy but reckless shepherd men According to unthink your sickness Come bring him not by him hither to his courtesy Be theme so true gentleness my fair and duty And thou so buried out Sennet They enter Your Majesty let 's means this marriag